This is an [jupyter](http://jupyter.org) notebook.
Lectures about Python, useful both for beginners and experts, can be found at http://scipy-lectures.github.io.

Open the notebook by (1) copying this file into a directory, (2) in that directory typing 
jupyter-notebook
and (3) selecting the notebook.

# <font color= 'blue'>Analysis of the 1-D Convolution Operation </font>
### <i>Practical Assignment 1, EE210 (Spring 2017) </i>
___
A notebook by ___Dhruv Ilesh Shah___ and ___Shashwat Shukla___  
___
__Required Packages:__ Python(2.7+), NumPy, math, cmath, SciPy.io _(Optional, to simplify IO operations)_
___
In this notebook, we present an analysis of the various algorithms for computing the one-dimensional convolution operation, along with their implementation and suitable improvisations. This includes the crude implementation, parallelised implementation, Cooley-Tukey algorithm (FFT), improvised Cooley-Tukey and a vectorised FFT-based implementation. We will look at the Python and C++ implementations on both CPU and GPU.

## Motivation
The convolution operation is a fundamental operation on two functions $f$ and $g$ which produces a third function, that is typically viewed as a modified version of one of the original functions, giving the integral of the pointwise multiplication of the two functions as a function of the amount that one of the original functions is translated. Convolution is similar to _cross-correlation_.

$$
(f*g)(t) \triangleq \int_{-\infty}^{\infty} f(\tau) g(t - \tau)
$$
Where $f, g : [0, \infty) \rightarrow \mathbb{R}$.
This can be extended to discrete-time signals as
$$
(f*g)[n] \triangleq \sum_{m = -\infty}^{\infty} f[m] g[n - m]
$$

For any physical _linear & time invariant (LTI)_ system $T$, it can be shown that the output of $T$ can be represented as $y(t) = x(t) * h(t)$, where $h(t)$ represents the response of $T$ to the unit impulse. This means that given an LTI system, the complete system can be represented solely by its impulse response $h(t)$. _(Similar definition for discrete-time signals.)_

## The Problem

Given a single-channel audio input $(16kHz)$, and the 2-channel impulse response of the surroundings as heard by the pair of human ears, generate a stereo reconstruction of the input.

$$
\begin{equation}
\begin{split}
y_l (t) &= x (t) * h_l (t) \\
y_r (t) &= x (t) * h_r (t)
\end{split}
\end{equation}
$$
***
In a typical scenario, the impulse response depends on the surroundings due to multiple reflections, interference and reverberation etc. The energy of the response, however, decays slowly and we have a finite duration response.
![Impulse Response of a Hall](http://www.prosoundweb.com/images/uploads/RationalGuideFigure1June2015.jpg)
<center>Courtesy: [ProSoundWeb](http://www.prosoundweb.com/)</center>

## Overview

As is clear from the definition, the convolution is an expensive operation and given the size of a typical sound clip $(10 sec \rightarrow 1.6\times10^5samples)$, the computation can be very intensive. But given the essentiality of the convolution as a fundamental property of an LTI system, and the immense number of applications, it cannot be ignored. Hence, we present an analysis of various ways to compute this convolution by employing certain other beautiful properties of this operation. We begin with algorithmic analysis for various methods to compute the convolution, and then go ahead to solve the actual problem with the best implementation. The topics covered are listed below:


1. Naive Implementation
2. Naive Implementation (parallelised)
3. Switching to the Frequency Domain  
      a. Naive Fast Fourier Transform  
      b. Cooley-Tukey Algorithm  
      c. Cooley-Tukey Algorithm (Improvisations)

Towards the end, we use an improvised version of the _Cooley-Tukey Algorithm_ to compute the linear convolution of the given signals.

# <font color=blue> Naive Implementation </font>

Given the input signal $x(t)$ and the left-ear impulse response, say, $h_l (t)$. Then
$$
\begin{split}
y_l[n] &= x [n] *h_l[n] \\
&= \sum_{m = -\infty}^{\infty} h[m]\cdot x[n - m]
\end{split}
$$

The simplest way ahead would be to compute the above summation (smartly manipulating the limits, though) using an iterative approach. The implementation is given below.

In [19]:
import numpy as np

# Create the signals to be convolved
source = np.random.random(5000)
fir = np.random.random(5000)

def naive_convolve(x, h):
    # Declaring the resultant, and allocating memory
    y = np.zeros(len(source) + len(fir) - 1)
    for i in range(len(x)):
        for j in range(len(h)):
            y[i+j] += (x[i] * h[j])
    return y

In [20]:
time naive_con = naive_convolve(source, fir)

CPU times: user 22.8 s, sys: 132 ms, total: 22.9 s
Wall time: 22.8 s


***
Clearly, we can see that $\approx 23s$ for convoluting $5000\times5000$ is very poor indeed, and can be a major bottleneck for the various applications dependent on the convolution operation. Complexity analysis for this clearly gives the time taken to be $\mathcal{O}(m n)$, or $\mathcal{O}(n^2)$ in general.

Extending this to the required application, where we would be convoluting $200000\times70000$ terms, extending the times and accounting for overheads gives an estimate of $\approx 3.5hr$. The above code took $212min$ to give the convoluted result on the given input. Clearly, this is not acceptable. So how do we improve this algorithm?

It is not hard to see that the computations in the outer loop are independent and hence can be run as independent threads on a parallelised processor or GPU. Given infinite cores, the execution time would be $\mathcal{O}(n)$. Practical figures are much lower this mark. This is the motivation behind the next approach.
***

# <font color=blue> Naive Implementati </font>